## Import all required python packages:

In [1]:
import psycopg2
import pandas as pd
import numpy as np 
import seaborn as sns
import cufflinks as cf
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

## Prepare parameters for to establish database connection and implement functions:

We are going to use PostgreSQL database with prepared schema coontaining all required data and analytical querries as views.

There are two main functions: <br>
-- <b>connect :</b> gets cridentials for database, establishes and returns a connection <br>
-- <b>postgresql_to_dataframe :</b> gets connection, query and column names and creates dataframe based on query result

In [2]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "localhost",
    "database"  : "space_missions_OLTP",
    "user"      : "postgres",
    "password"  : "xiaomi1919"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

## Establish and Test connection:

In [3]:
# Connect to the database
conn = connect(param_dic)

column_names = ["company_id", "company", "created_at"]
# Execute the "SELECT" query
df_companies = postgresql_to_dataframe(conn, "select * from sp_missions.company", column_names)
# Close the connection
conn.close()

Connecting to the PostgreSQL database...
Connection successful


In [4]:
df_companies.head(5)

,company_id,company,created_at
0,1,RAE,2022-01-20
1,2,Khrunichev,2022-01-20
2,3,Lockheed,2022-01-20
3,4,Exos,2022-01-20
4,5,ILS,2022-01-20


We have successfuly connected to data source and fetched data from table.

## TASK DESCRIPTION:

Our telecom company based in Uzbekistan has a new satellite to increase coverage for our services. We have to send it into orbit as soon as possible with lowest possible price. We need to find a company that will get the contract for this mission. In order to find out the company we have to make analysis on provided data. 

### Q1) IN WHICH MONTH THE LAUNCH HAS HIGHEST POSSIBILITY TO SUCCEED:

First thing we have to find the date for our mission. For that we need to find in which month the lauch has the highest probability to succeed.

#### SQL QUERY

In [5]:
# select CASE extract(month from m.mission_date)
#            WHEN 1 THEN 'January'
#            WHEN 2 THEN 'February'
#            WHEN 3 THEN 'March'
#            WHEN 4 THEN 'April'
#            WHEN 5 THEN 'May'
#            WHEN 6 THEN 'June'
#            WHEN 7 THEN 'July'
#            WHEN 8 THEN 'August'
#            WHEN 9 THEN 'September'
#            WHEN 10 THEN 'October'
#            WHEN 11 THEN 'November'
#            WHEN 12 THEN 'December'
#        END as month,
#        count(mission_id) as total_success_missions,
#        sum(count(mission_id)) over () as tatal_missions,
#        round(count(mission_id) / sum(count(mission_id)) over () * 100, 2) as percentage
# from mission m
# where m.status = 'Success'
# group by extract(month from m.mission_date)
# order by percentage asc;

#### DATAFRAME

In [6]:
# Connect to the database
conn = connect(param_dic)

column_names = ["month", "total_success_missions", "total_missions", "percentage"]
# Execute the "SELECT" query
df_month_data = postgresql_to_dataframe(conn, "select * from sp_missions.success_mission_by_month", column_names)
# Close the connection
conn.close()
# display dataframe:
df_companies.head(5)

Connecting to the PostgreSQL database...
Connection successful


,company_id,company,created_at
0,1,RAE,2022-01-20
1,2,Khrunichev,2022-01-20
2,3,Lockheed,2022-01-20
3,4,Exos,2022-01-20
4,5,ILS,2022-01-20


#### PLOT

In [7]:
fig = px.histogram(df_month_data, x="month", y="total_success_missions", title='Success missions by month', 
                   marginal='box', color_discrete_sequence=['lightblue'])

fig.update_layout(
    xaxis_title_text='Month',
    yaxis_title_text='Missions',
    bargap=0.1
)

#### RESULT:
We can clearly see that we should shedule our launch to December as it will have the highest chance to succeed.

### Q2) TOP 10 COMPANIES BY NUMBER OF LAUNCHES:

We first can look for top companies by number of launches. More launches means more experience. From this analysis stage we can find our list of candidate companies.

#### SQL QUERY

In [45]:
# with cte_launch_data as (
#     select company,
#            status,
#            total_missions_by_status,
#            total_missions_by_company,
#            dense_rank()
#            over (order by total_missions_by_company desc, company 
#            groups between unbounded preceding and unbounded following) as rank
#     from (
#              select c.company,
#                     m.status,
#                     count(m.mission_id)                                    as total_missions_by_status,
#                     sum(count(m.mission_id)) over (partition by c.company) as total_missions_by_company
#              from mission m
#                       inner join company c on m.company_id = c.company_id
#              group by c.company, m.status) t
# ) select company, status, total_missions_by_status, total_missions_by_company from cte_launch_data where rank <= 10;

#### DATAFRAME

In [46]:
# Connect to the database
conn = connect(param_dic)

column_names = ["company", "status", "total_missions_by_status", "total_missions_by_company"]
# Execute the "SELECT" query
df_launches = postgresql_to_dataframe(conn, "select * from sp_missions.launches_by_company", column_names)
# Close the connection
conn.close()
# display dataframe:
df_by_company = df_launches[["company", "total_missions_by_company"]].groupby('company').first().reset_index().sort_values(by=['total_missions_by_company'], ascending=False)
df_launches.head(5)

Connecting to the PostgreSQL database...
Connection successful


,company,status,total_missions_by_status,total_missions_by_company
0,RVSN USSR,Failure,121,1777
1,RVSN USSR,Prelaunch Failure,1,1777
2,RVSN USSR,Partial Failure,41,1777
3,RVSN USSR,Success,1614,1777
4,Arianespace,Partial Failure,3,279


#### PLOT

In [51]:
fig = px.pie(df_by_company, values='total_missions_by_company',
             names='company', title='# of missions by company',
            color_discrete_sequence=px.colors.sequential.ice)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=2)))
fig.show()

#### RESULT:
We can see that we have 10 companies as RVSN USSR with 1777 launches (highest) and Martin Marietta with 114 launches (lowest). 

We can further analyse these companies, however we know that USSR does not exists now and such RVSN. In addition some companies might not have operational rockets. 
<br> <br>
We need a better method for finding our target companies.

### Q3) TOP 5 COMPANIES BY YEARLY INCREASE IN 5 YEAR PERIOD:

What we do here is that we first take companies that have active rockets ready for launch. <br>
We than calculate difference beetween success launches and failed launches to find absolute success factor of company. <br>
Lastly, we find yarly increase and it's total and take top 5 companies by amount of increase.

#### SQL QUERY

In [ ]:
# with cte_active_companies as (
#      select distinct c.company_id, c.company
#         from company c
#         inner join mission m on c.company_id = m.company_id
#         inner join rocket r on r.rocket_id = m.rocket_id
#         where r.is_rocket_active = true
# ), cte_yearly_stats as (
# select extract(year from m.mission_date) as year,
#        ac.company,
#        count(m.mission_id) filter ( where m.status = 'Success' ) as succes_missions,
#        count(m.mission_id) filter ( where m.status in ('Partial Failure', 'Failure', 'Prelaunch Failure') ) 
#     as failed_missions
# from  mission m
# inner join cte_active_companies ac on ac.company_id = m.company_id
# where extract(year from mission_date) between (select extract(year from max(mission_date)) - 5 from mission)
#                                               and (select extract(year from max(mission_date)) from mission)
# group by extract(year from m.mission_date), ac.company
# order by company
# ), cte_change_stats as (
#     select year,
#        company,
#        succes_missions - failed_missions as positive_mission_count,
#        (succes_missions - failed_missions) - first_value(succes_missions - failed_missions) 
#     over (partition by company order by year) as change_from_first_year
# from cte_yearly_stats
# ), cte_total_changee as (
#     select year,
#        company,
#        positive_mission_count,
#        change_from_first_year,
#        sum(change_from_first_year) over (partition by company) as total_increase
#     from cte_change_stats
# ), cte_ranking as (
#    select cth.year,
#           cth.company,
#           cth.positive_mission_count,
#           cth.change_from_first_year,
#           cth.total_increase,
#           dense_rank() over (order by total_increase desc, company 
#                              groups between unbounded preceding and unbounded following) as rank
#     from cte_total_changee cth
# ) select year,
#          company,
#          positive_mission_count,
#          change_from_first_year,
#          total_increase
# from cte_ranking
# where rank <= 5
# and year  between (select extract(year from max(mission_date)) - 4 from mission)
#                                               and (select extract(year from max(mission_date)) from mission)
# order by company, year;

#### DATAFRAME

In [62]:
# Connect to the database
conn = connect(param_dic)

column_names = ["year", "company", "positive_mission_count", "change_from_first_year", "total_increase"]
# Execute the "SELECT" query
df_company_stats = postgresql_to_dataframe(conn, "select * from sp_missions.company_rank_by_increase", column_names)
# Close the connection
conn.close()
# display dataframe:
df_company_stats.head(5)

Connecting to the PostgreSQL database...
Connection successful


,year,company,positive_mission_count,change_from_first_year,total_increase
0,2016,CASC,18,16,97
1,2017,CASC,12,10,97
2,2018,CASC,37,35,97
3,2019,CASC,25,23,97
4,2020,CASC,15,13,97


#### PLOT

In [81]:
# Allows us to create graph objects for making more customized plots
import plotly.graph_objects as go

fig = px.line(df_company_stats, x='year', y='positive_mission_count', color='company',
              symbol="company", title = '# of positive missions by company')
fig.show()

In [86]:
fig = px.density_heatmap(df_company_stats.sort_values(by=['change_from_first_year'], ascending=False), 
                         x='year', y='company', z='change_from_first_year', 
                         
                         title = 'year vs change in positive mission count',
                         nbinsx=5, nbinsy=5, text_auto=True)
fig

#### RESULT:
From line chart we can see that <b>CASC</b> and <b>Space X</b> both have highes positive launch history. In addition, Space X in 2020 approaches CASC and has positive slope that indicates rise of performance. <br>
<br>
Furthermore, from Heatmap we see that again <b>CASC</b> and <b>Space X</b> both have the highest increase from 2016 in positive launch counts reaching 13 and 9 resprectively. <br>
<br>
Based on results above, I have decided to condsider only Space X and CASC for further analysis.

### Q4) MISSION COST FOR SpaceX AND CASC:

On this stage we have only 2 candidate companies left. <br>
As stated in task description, we have to send our payload as cheap as possible.
<br>
In this analysis step, we will compare mission costs for both companies. 

#### SQL QUERY

In [ ]:
# CREATE VIEW company_mission_cost AS
# select c.company,
#        ml.mission_location,
#        extract(year from mission_date) as year,
#        r.rocket,
#        avg(m.cost) as average_mission_cost
# from mission m
# inner join company c on c.company_id = m.company_id
# inner join rocket r on m.rocket_id = r.rocket_id
# inner join mission_location ml on m.mission_location_id = ml.mission_location_id
# where company in ('CASC', 'SpaceX')
# and m.cost is not null
# and r.is_rocket_active = true
# group by c.company, ml.mission_location,  extract(year from mission_date), r.rocket;

#### DATAFRAME

In [108]:
# Connect to the database
conn = connect(param_dic)

column_names = ["company", "mission_location", "year", "rocket", "average_mission_cost"]
# Execute the "SELECT" query
df_company_cost = postgresql_to_dataframe(conn, "select * from sp_missions.company_mission_cost", column_names)
# Close the connection
conn.close()
# display dataframe:
df_company_cost.head(5)

Connecting to the PostgreSQL database...
Connection successful


,company,mission_location,year,rocket,average_mission_cost
0,CASC,China,1982,Long March 2C,30.8000000000000000
1,CASC,China,1983,Long March 2C,30.8000000000000000
2,CASC,China,1984,Long March 2C,30.8000000000000000
3,CASC,China,1985,Long March 2C,30.8000000000000000
4,CASC,China,1986,Long March 2C,30.8000000000000000


#### PLOT by Location

In [123]:
fig = px.scatter_3d(df_company_cost, x='company', y='mission_location', z='average_mission_cost', 
                    color='company')
fig.update_traces(marker=dict(size=5))
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig.show()

#### PLOT by Rocket

In [125]:
fig = px.scatter_3d(df_company_cost, x='company', y='rocket', z='average_mission_cost', 
                    color='rocket')
fig.update_traces(marker=dict(size=5))
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig.show()

#### RESULT:
From above 3d scatter plots we can see that we have two option in terms of country namely China and USA. As we are located in Uzbekistan, it seems that China is a good option for us in terms of logistics. It's costly to transfer our payload to USA. <br>
<br>

However, In terms of rockets and overall cost, Falcon 9 by Space X is a best option in terms of price (50 millions) compared to Long March 4C and 4B (64.88 millions). I am comparing only these to rockets because they are all in the same class and are best suited for our task.

## OVERALL:

Based on above research and analysis, we would prefer SpaceX as our target company for our mission. Decision was made based on overall performance and cost.Moreover, based on my own expericence, Falcon 9 is much advanced in terms of technology.